## Random Forest

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
features= pd.read_csv('data/temps.csv')
features.head(5)

In [2]:
features.shape

NameError: name 'features' is not defined

In [3]:
features.describe()

NameError: name 'features' is not defined

In [4]:
### Data Preparation - One hot encoding
features = pd.get_dummies(features)

features.iloc[:, 5:].head(5)

NameError: name 'features' is not defined

In [6]:
# Convert data into arrays
labels = np.array(features['actual'])

#Remove labels from features
features = features.drop('actual', axis=1)

#SAve feature names for later use
feature_list = list(features.columns)

#Convert to numpy array
features = np.array(features)

NameError: name 'features' is not defined

In [8]:
## Training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size=0.25, random_state=42)

NameError: name 'features' is not defined

In [9]:
# EStablish baseline
baseline_preds = test_features[:, feature_list.index('average')]

#Baseline error
baseline_errors = abs(baseline_preds - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

NameError: name 'test_features' is not defined

In [10]:
#Train model
rf = RandomForestRegressor(n_estimators = 1000, random_state=42)
rf.fit(train_features, train_labels)

NameError: name 'RandomForestRegressor' is not defined

In [11]:
## Make predictions

predictions = rf.predict(test_features)
errors = abs(predictions - test_labels)
print('Mean Absolute error: ', round(np.mean(errors), 2), 'degrees.')

NameError: name 'rf' is not defined

In [12]:
## Determine Performance Metrics
mape = 100 * (errors/test_labels)

accuracy = 100 - np.mean(mape)
print('Accuracy: ', round(accuracy, 2), '%')

NameError: name 'errors' is not defined

### Visualise the trees

In [15]:
import pydot
from sklearn.tree import export_graphviz

In [16]:
#Pull one tree from forest
tree = rf.estimators_[5]

#Export image to dot file
export_graphviz(tree, out_file='tree.dot', feature_names = feature_list,
               rounded=True, precision = 1)

#Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

#Write graph to PNG
graph.write_png('tree.png')

NameError: name 'rf' is not defined

In [17]:
# Limit depth of tree to 3 levels

rf_small = RandomForestRegressor(n_estimators=10, max_depth=3)
rf_small.fit(train_features, train_labels)
#Extract the small tree
tree_small = rf_small.estimators_[5]

export_graphviz(tree_small, out_file='small_tree.dot', feature_names = feature_list,
               rounded=True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('tree_small.dot')
graph.write_png('tree_small.png')

NameError: name 'RandomForestRegressor' is not defined

### Variable Importances

In [18]:
#Get numeric feature importance
importances = list(rf.feature_importances_)

#List of tuples with variable and importance
feature_importances = [(feature, round(importance,2)) for feature, importance in zip(feature_list, importances)]

#Sort feature importances by most important first
feature_importances = sorted(feature_importances, key=lambda x:x[1], reverse=True)

#Print feature and importance
[print("Variable: {:20} Importance: {}".format(*pair)) for pair in feature_importances];

NameError: name 'rf' is not defined

In [19]:
## Random Forest with 2 most important variables
rf_most_important = RandomForestRegressor(n_estimators = 1000, random_state=42)

#Extract 2 most imp features
important_indices = [feature_list.index('temp_1'), feature_list.index('average')]
train_important = train_features[:, important_indices]
test_important = test_features[:, important_indices]

#Train RF
rf_most_important.fit(train_important, train_labels)

#Predict
predictions = rf_most_important.predict(test_important)
errors = abs(predictions - test_labels)

#Metrics
print('Mean Absolute Error: ', round(np.mean(errors), 2), 'degrees.')
mape = np.mean(100 * (errors / test_labels))
accuracy = 100 - mape
print('Accuracy: ', round(accuracy, 2), '%')


NameError: name 'RandomForestRegressor' is not defined

### Visualise Importance

In [20]:
plt.style.use('fivethirtyeight')
x_values = list(range(len(importances)))

plt.bar(x_values, importances, orientation='vertical')
plt.xticks(x_values, feature_list, rotation='vertical')

plt.ylabel('Importance')
plt.xlabel('Variable')
plt.title('Variable Importance')

NameError: name 'importances' is not defined